In [1]:
DATASET_PATH = 'https://cdn.stackoverflow.co/files/jo7n4k8s/production/49915bfd46d0902c3564fd9a06b509d08a20488c.zip/stack-overflow-developer-survey-2023.zip'

In [106]:
DATA_PATH = '/content/survey_results_public.csv'
EXPORT_PATH = '/content/01_preprocessed_data.pickle'

In [52]:
REPLACE_CAT = {
   'YearsCodePro': {'Less than 1 year':'1','More than 50 years':'51'},
   'YearsCode':  {'Less than 1 year':'1','More than 50 years':'51'},
}

In [53]:
!wget $DATASET_PATH

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [54]:
!unzip stack-overflow-developer-survey-2023.zip

Archive:  stack-overflow-developer-survey-2023.zip
replace README_2023.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: README_2023.txt         
replace so_survey_2023.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: so_survey_2023.pdf      
  inflating: survey_results_public.csv  
  inflating: survey_results_schema.csv  


In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [83]:
raw_df = pd.read_csv(DATA_PATH)

In [84]:
raw_df.head()

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I agree,None of these,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,285000.0
2,3,I agree,I am a developer by profession,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,...,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,250000.0
3,4,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,Automated testing;Continuous integration (CI) ...,NaN,Appropriate in length,Easy,156000.0
4,5,I agree,I am a developer by profession,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,...,1-2 times a week,1-2 times a week,3-5 times a week,60-120 minutes a day,30-60 minutes a day,Microservices;Automated testing;Observability ...,Other,Appropriate in length,Neither easy nor difficult,23456.0


In [85]:
survey_df = raw_df.copy()

In [86]:
years_exp = ['YearsCodePro', 'YearsCode']

for col in years_exp:
    print(col)
    print(survey_df[col].unique().tolist())
    print(len(survey_df[col].unique().tolist())*'-----')
    print()

YearsCodePro
[nan, '9', '23', '7', '4', '21', '3', '15', 'Less than 1 year', '10', '2', '6', '14', '5', '19', '13', '16', '28', '1', '30', '11', '8', '25', '32', '24', '40', '17', '45', '29', '12', '31', '20', '18', '50', '27', '43', '22', '26', '38', '33', '44', '35', '34', '37', '42', '41', 'More than 50 years', '47', '36', '39', '48', '46', '49']
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

YearsCode
[nan, '18', '27', '12', '6', '21', '4', '5', '20', '14', '10', '15', '11', '3', '24', '8', '13', 'Less than 1 year', '16', '33', '22', '30', '32', '7', '35', '28', '40', '17', '29', '19', 'More than 50 years', '9', '38', '26', '34', '25', '2', '45', '23', '31', '43', '1', '48', '41', '50', '39', '42', '37', '36', '44', '46', '49', '47']
--------------------------------

In [92]:
def split_answers(data_series, delimiter=";"):


    def is_splittable(pd_series, delimiter):

        return pd_series.str.contains(delimiter)

    def split_answer(pd_series, delimiter):

        return pd_series.str.split(delimiter)


    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series

    modified_series = split_answer(data_series, delimiter)

    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])

    return modified_series

In [88]:
for col , replacement in REPLACE_CAT.items():
    survey_df[col] = survey_df[col].replace(replacement).astype(np.float64)

In [89]:
obj_cols = survey_df.select_dtypes(include=['object']).columns
for col in obj_cols:
    survey_df[col] = split_answers(survey_df[col])

In [93]:
i = survey_df.sample(1).index[0]
print(survey_df['LanguageHaveWorkedWith'].iloc[i])
print(raw_df['LanguageHaveWorkedWith'].iloc[i])

['Bash/Shell (all shells)', 'C#', 'HTML/CSS', 'JavaScript', 'Kotlin', 'Python']
Bash/Shell (all shells);C#;HTML/CSS;JavaScript;Kotlin;Python


In [97]:
i = survey_df.sample(1).index[0]
print(survey_df['DevType'].iloc[i])
print(raw_df['DevType'].iloc[i])

DevOps specialist
DevOps specialist


In [105]:
for col in obj_cols:
    i = survey_df.sample(1).index[0]
    print(survey_df[col].iloc[i])
    print(raw_df[col].iloc[i])
    print('---------------------------------')
    print()

I agree
I agree
---------------------------------

I am a developer by profession
I am a developer by profession
---------------------------------

25-34 years old
25-34 years old
---------------------------------

['Not employed, but looking for work']
Not employed, but looking for work
---------------------------------

Hybrid (some remote, some in-person)
Hybrid (some remote, some in-person)
---------------------------------

[]
nan
---------------------------------

Master’s degree (M.A., M.S., M.Eng., MBA, etc.)
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)
---------------------------------

['Books / Physical media', 'Online Courses or Certification', 'Other online resources (e.g., videos, blogs, forum)']
Books / Physical media;Online Courses or Certification;Other online resources (e.g., videos, blogs, forum)
---------------------------------

['Formal documentation provided by the owner of the tech', 'Blogs with tips and tricks', 'Books', 'Recorded coding sessions', 'Auditory

In [107]:
survey_df.to_pickle(EXPORT_PATH)